In [17]:
from Bio import SearchIO
from Bio import SeqIO
from Bio import Phylo
from Bio.Seq import Seq
from Bio import AlignIO

from io import StringIO
import subprocess

from os import listdir
from os.path import isfile, join, dirname, isdir, exists
from os import path
import os

import matplotlib.pyplot as plt
import numpy as np

In [18]:
#These are the queries for the hmmsearch command. Change the genus or domain to your liking

genus = 'magnaporthe' #or try 'neurospora'
domain = 'NACHT' #try 'NACHT', 'NB-ARC', or 'HET'

In [23]:
#helper function for preparing directories. do not change this.

def make_path(*argv):
    mypath = path.join(*argv)
    if not path.exists(dirname(mypath)):
        os.mkdir(dirname(mypath))
    return mypath

def make_dir(*argv):
    mydir = path.join(*argv)
    if not path.exists(mydir):
        os.mkdir(mydir)
    return mydir


In [27]:
#preparing path names for hmmsearch command...
proteome_path = make_path('proteomes', genus, os.listdir(make_dir('proteomes', genus))[0])
genomes_dir = make_dir('genomes', genus)
assembly_name = proteome_path.split('_')[-2]
domain_path = make_path('pfam_domains', domain+'.hmm')
hmmsearch_align_path = make_path('hmmsearch_align', genus, domain+'-'+assembly_name+'.aln')
hmmsearch_out_path = make_path('hmmsearch_out', genus, domain+'-'+assembly_name+'.out')
hmmmsearch_domout_path = make_path('hmmsearch_out', genus, domain+'-'+assembly_name+'.domout')

In [36]:
!hmmsearch --domE 0.01  -A $hmmsearch_align_path --domtblout $hmmmsearch_domout_path $domain_path $proteome_path > $hmmsearch_out_path

In [28]:
#Parse hmmsearch output and filter hits by e-value. Find these hits the reference proteome and display annotations of the numTop proteins along with their E-values

numTop = 35 ## change this to adjust the number of protein descriptions displayed
ethresh = 0.01 ### adjust this for e-value threshold

qresult = next(SearchIO.parse(hmmmsearch_domout_path, 'hmmsearch3-domtab'))
filtered_hits = list(filter(lambda hsp: hsp.evalue < ethresh, qresult.hsps))
hit_ids = [hsp.hit_id for hsp in filtered_hits]
hit_sequences = sorted([record for record in SeqIO.parse(proteome_path, "fasta") if record.id in hit_ids], key=lambda a: hit_ids.index(a.id))
print('\n'.join([f'E-value: {filtered_hits[ii].evalue} for ' + hit_sequences[ii].description for ii in range(numTop)]))

E-value: 1e-13 for XP_003720097.1 uncharacterized protein MGG_17753 [Pyricularia oryzae 70-15]
E-value: 1.3e-11 for XP_003711083.1 ankyrin repeatl protein [Pyricularia oryzae 70-15]
E-value: 9.9e-10 for XP_003718688.1 uncharacterized protein MGG_00388 [Pyricularia oryzae 70-15]
E-value: 3.1e-09 for XP_003712653.1 uncharacterized protein MGG_05125 [Pyricularia oryzae 70-15]
E-value: 3.5e-08 for XP_003712408.1 uncharacterized protein MGG_04917 [Pyricularia oryzae 70-15]
E-value: 3.3e-09 for XP_003713330.1 uncharacterized protein MGG_15509 [Pyricularia oryzae 70-15]
E-value: 1.5e-08 for XP_003713943.1 uncharacterized protein MGG_08914 [Pyricularia oryzae 70-15]
E-value: 3.3e-08 for XP_003720118.1 uncharacterized protein MGG_12047 [Pyricularia oryzae 70-15]
E-value: 2.7e-08 for XP_003710235.1 ankyrin repeat domain-containing protein 29 [Pyricularia oryzae 70-15]
E-value: 9.4e-08 for XP_003721297.1 NACHT and Ankyrin domain-containing protein [Pyricularia oryzae 70-15]
E-value: 2.5e-07 for X

In [29]:
##Exonerates a protein to all genomes for the species. don't change this.
def exonerate(nlr_protein):    
    nlr_path = make_path('nlr_protein', genus, 'nlr_'+nlr_protein.id+'.fasta')
    p2gaa_path = make_path('exonerate_p2gaa', genus, 'p2gaa_%s.faa' % nlr_protein.id)
    p2gnt_path = make_path('exonerate_p2gnt', genus, 'p2gnt_%s.fasta' % nlr_protein.id)
    SeqIO.write(nlr_protein, nlr_path, "fasta")
    
    files = [f for f in listdir(genomes_dir) if isfile(join(genomes_dir, f)) and f != '.DS_Store' ]
    numExonerated = 0

    for ii, f in enumerate(files):
        proc = subprocess.run(['exonerate', '--model', 'protein2genome', '--bestn', '1', '--showvulgar', 'F', '--ryo', '''">%ti (%tab - %tae)\n%tas\n"''', nlr_path, join(genomes_dir, f)],  stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        outsplit = proc.stdout.decode().split('''"''')
        if len(outsplit) > 3:
#             print('exonerating', f, 'counter: ', ii)
            numExonerated += 1

            qresult = next(SearchIO.parse(StringIO(outsplit[-3]), 'exonerate-text'))
            aa_record = sum([frag.hit for frag in qresult[0][0]], Seq(''))

            if not ii:
                writeMode = 'w'
            else:
                writeMode = 'a'
            
            aafile = open(p2gaa_path, writeMode)        
            SeqIO.write(aa_record, aafile, 'fasta')
            aafile.close()

            ntfile = open(p2gnt_path, writeMode)        
            ntseq = outsplit[-2]
            ntfile.write(ntseq)
            ntfile.close()
            
            if not ii % 50:
                print(ii+1, 'sequences exonerated')
#         else:
#             print('failed to exonerate', f, 'counter: ', ii)

    return numExonerated, ii+1


#get column of position weight matrix
def get_distr(aa_col, pseudocounts = 0.0):
    abundict = {aa:pseudocounts for aa in list('ACDEFGHIKLMNPQRSTVWY-X*')}
    for site in aa_col:
        if site in abundict.keys():
            abundict[site] += 1.0
        else:
            print('did not find', site, 'in alphabet')
            raise Exception()
    abundict = {aa:count for aa,count in abundict.items() if count > 0.0}
    weights = np.array(list(abundict.values()))
    distr = weights/sum(weights)
    return distr

#compute entropy from a list of weights
def get_entropy(distr):
    return -np.sum(np.log(distr)*distr)

#returns entropy sequence from multiple alignment
def get_entropy_sequence(align):
    entropies = []
    for col in range(len(align[0,:])):
        entropies.append(get_entropy(get_distr(align[:, col])))
    return np.array(entropies)

#plot entropy sequence
def plot_entropy(entropies, protein_id):
    plt.plot(entropies)
    plt.xlabel('Amino acid position')
    plt.ylabel('Entropy')
    plt.title('Splice-aware entropy of %s' % protein_id)
    
    
# def save_plot(nlr_protein):    
#     entropy_seq_path = make_path('entropy', genus, 'seq', 'entropy_clustalo_%s.npy' % nlr_protein.id)
#     plot_entropy(np.load(entropy_seq_path), nlr_protein.id)
#     entropy_plot_path = make_path('entropy', genus, 'plot', 'entropy_clustalo_%s.png' % nlr_protein.id)
#     plt.savefig(entropy_plot_path)    
    
def get_multiple_alignment(nlr_protein):
    print('exonerating protein', nlr_protein.id, '...')
    numExonerated, numGenomes = exonerate(nlr_protein)
    print(numExonerated, 'genomes out of', numGenomes, 'total contain an ortholog of protein', nlr_protein)
    
    p2gaa_path = make_path('exonerate_p2gaa', genus, 'p2gaa_%s.faa' % nlr_protein.id)
    clustalo_out_path = make_path('clustalo_out', genus, 'clustalo_%s.out' % nlr_protein.id)
    !clustalo -i $p2gaa_path -o $clustalo_out_path
    
def save_entropy(nlr_protein):
    clustalo_out_path = make_path('clustalo_out', genus, 'clustalo_%s.out' % nlr_protein.id)
    entropies = get_entropy_sequence(AlignIO.read(clustalo_out_path, "fasta"))
    entropy_seq_path = make_path('entropy', genus, 'seq', 'entropy_clustalo_%s.npy' % nlr_protein.id)    
    np.save(entropy_seq_path, entropies)
    entropy_plot_path = make_path('entropy', genus, 'plot', 'entropy_clustalo_%s.png' % nlr_protein.id)
    plot_entropy(entropies, nlr_protein.id)
    plt.savefig(entropy_plot_path)
    plt.close()

def main(nlr_protein):
    get_multiple_alignment(nlr_protein)
    save_entropy(nlr_protein)
    
def loop_over_hits(hits):
    for nlr_protein in hits:
        main(nlr_protein)
    print('completed pipeline on protein', nlr_protein.id)
    


In [15]:
# use the main function to exonerate a protein against the genomes,
# compute multiple alignment, and save entropy plots
# also try loop_over_hits(hit_sequences) to run the pipeline on
# a list hit sequences found by hmmer

main(hit_sequences[3])

exonerating protein XP_003712653.1 ...
1 sequences exonerated
51 sequences exonerated
101 sequences exonerated
151 sequences exonerated
201 sequences exonerated
250 genomes out of 250 total contain an ortholog of protein ID: XP_003712653.1
Name: XP_003712653.1
Description: XP_003712653.1 uncharacterized protein MGG_05125 [Pyricularia oryzae 70-15]
Number of features: 0
Seq('MPSNSEYTIGWIAAIETEFVAASRFLDKRHGRPESQDSADSNSYKLGSMEGHNV...RFV')
FATAL: Cowardly refusing to overwrite already existing file 'clustalo_out/magnaporthe/clustalo_XP_003712653.1.out'. Use --force to force overwriting.
